In [ ]:
def strategy(df, trigger, window=1):
    df = df.copy()
    df['log_ret'] = df.Close.apply(np.log).diff()
    df['prior_n'] = df.log_ret.rolling(window).sum()
    df.dropna(inplace = True)

    df['position'] = [1 if i>trigger else 0 for i in df.prior_n] 
    ## Long, short or neutral
    
    df['log_strat'] = df.position.shift(1) * df.log_ret

    # number of trades
    df['trades'] = 0
    trades = 0
    for i in range(len(df['position']) - 1):
        if df['position'].iloc[i] != df['position'].iloc[i+1] : 
            trades += 1
        df.iloc[i+1, df.columns.get_loc('trades')] = trades

    return df

In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
df = yf.download('BTC-USD', start = '2022-12-3', interval = '1d',auto_adjust=True)
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2022-12-03,16908.236328,17116.041016,16888.140625,17090.097656,16217776704
2022-12-04,17130.486328,17157.767578,16903.439453,16908.169922,16824520830
2022-12-05,16974.826172,17378.152344,16922.431641,17128.894531,22209086834
2022-12-06,17089.503906,17091.863281,16939.921875,16975.238281,19889922369
2022-12-07,16848.126953,17109.376953,16750.558594,17089.505859,19675404389
...,...,...,...,...,...
2025-02-15,97580.351562,97975.039062,97240.195312,97508.382812,17047266288
2025-02-16,96175.031250,97725.593750,96060.976562,97580.492188,16536755396


In [6]:
results = []#pd.DataFrame(columns = ['trigger', 'window', 'return', 'volatility' , 'trades' , 'asset return' , 'asset vol'])

for trigger in np.linspace(0.001, 0.01, 5):
    for window in np.linspace( 3 , 11, 6, dtype=int):
        df_i = strategy(df, trigger, window)
      
        vol = df_i[['log_strat']].std()*len(df)**.5 
        avol = df_i[['log_ret']].std()*len(df)**.5 
        strat_ret = np.exp( df_i['log_strat'].sum() ) - 1 
        asset_ret = np.exp( df_i['log_ret'].sum() ) - 1 
    
        #results.loc[len(results)] = [round(trigger, 4), int(window), float(strat_ret), float(vol.iloc[0]), df_i.trades.iloc[-1],                                     float(asset_ret), float(avol.iloc[0])]
        results.append({
            'trigger': round(trigger, 4),
            'window': int(window),
            'return': float(strat_ret),
            'volatility': float(vol.iloc[0]),
            'trades': df_i.trades.iloc[-1],
            'asset return': float(asset_ret),
            'asset vol': float(avol.iloc[0])
        })

    # print('Trigger ', round(trigger,4))

results = pd.DataFrame(results)

print(results)


    trigger  window    return  volatility  trades  asset return  asset vol
0    0.0010       3  1.053035    0.702612     282      4.694324   0.706660
1    0.0010       4  1.303266    0.700084     213      4.656113   0.707084
2    0.0010       6  0.164038    0.703000     177      4.608861   0.707474
3    0.0010       7  0.077131    0.705076     151      4.641703   0.707870
4    0.0010       9 -0.096540    0.709125     128      4.651255   0.708741
5    0.0010      11 -0.302263    0.709167     139      4.436052   0.708942
6    0.0032       3  0.810569    0.677713     307      4.694324   0.706660
7    0.0032       4  1.647769    0.679287     251      4.656113   0.707084
8    0.0032       6  0.032280    0.694468     203      4.608861   0.707474
9    0.0032       7 -0.006684    0.698669     178      4.641703   0.707870
10   0.0032       9 -0.166381    0.701680     145      4.651255   0.708741
11   0.0032      11 -0.298400    0.701416     162      4.436052   0.708942
12   0.0055       3  0.98